# Initial Art scrape

This folder is an offshoot of the music scrape so I could make the 3 column dataframes separately from the problematic scrapes and then merge them together in a new folder.  I didn't think through choosing text heavy posts, but it will be interesting to see if I can still make decent predictions.  I will also try scraping additional days to add more meagre posts, mostly for the title material it seems.

In [1]:
import requests
import time
import pandas as pd

In [2]:
#get an art reddit url
url = 'https://www.reddit.com/r/StreetArt.json'

headers ={'User-agent': 'pony-bot'}

res =requests.get(url, headers=headers)

In [5]:
res.status_code

200

In [6]:
art =res.json()

In [8]:
aurl ='https://www.reddit.com/r/StreetArt.json'

art_posts =[]
after =None
for i in range(3000):
   # print(i)
    if after ==None:
        params ={}
    else:
        params = {'after': after}
    res =requests.get(aurl, params =params, headers ={'User-agent': 'pony bot'})
    if res.status_code ==200:
              
        art =res.json()
        art_posts.extend(art['data']['children'])
        after = art['data']['after']
    time.sleep(2)
    
#print(len(posts))
#print(len({posts[i]["data"]["title"] for i in range(len(posts))}))

In [115]:
len(art_posts)

74774

In [2]:
#curious about uniqueness, although this always seems so bad.
artlist = []
for i in range(74774):
    artlist.append(art_posts[i]['data']['name'])
print(len(artlist))
len(set(artlist))

In [10]:
#make a copy just because I am paranoid
artlist =art_posts

In [16]:
#need to get rid of the 'kind', I need to look at the df to see what is going on.
for i in range(len(art_posts)):
        art_posts[i] = art_posts[i]["data"]

In [17]:
#make a df
art_df =pd.DataFrame(art_posts)

In [19]:
#check the shape, although I'm still scared I messed up the scrape.
art_df.shape

(74774, 99)

In [30]:
#export to a csv in case I destroy it.  Sunday morning
art_df.to_csv("../data/scrapes/art_csv")

# Read back in art csv and condense
I went from 74,774 posts to 998 in the snap of a finger, and then to 959 of those being null.

In [87]:
df =pd.read_csv('../data/scrapes/art_csv')

In [88]:
#look at the actual titles so I can select the ones I want to view.
df.shape

(74774, 100)

In [110]:
#df.name.value_counts()

In [124]:
pd.set_option('max_columns', 200)
df.head(2)

,Unnamed: 0,approved_at_utc,approved_by,archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,banned_at_utc,banned_by,can_gild,can_mod_post,category,clicked,content_categories,contest_mode,created,created_utc,crosspost_parent,crosspost_parent_list,distinguished,domain,downs,edited,gilded,gildings,hidden,hide_score,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,likes,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,media_embed,media_metadata,media_only,mod_note,mod_reason_by,mod_reason_title,mod_reports,name,no_follow,num_comments,num_crossposts,num_reports,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,quarantine,removal_reason,report_reasons,saved,score,secure_media,secure_media_embed,selftext,selftext_html,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,ups,url,user_reports,view_count,visited,whitelist_status,wls,words
0,0,NaN,NaN,False,deportambleme,NaN,NaN,NaN,[],NaN,NaN,NaN,text,t2_3icqxnkw,False,NaN,NaN,False,False,NaN,False,NaN,False,1.553970e+09,1.553970e+09,NaN,NaN,NaN,i.redd.it,0,False,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",False,False,b7e92j,False,False,False,True,True,False,False,NaN,NaN,NaN,[],NaN,dark,text,False,NaN,{},NaN,False,NaN,NaN,NaN,[],t3_b7e92j,False,3,0,NaN,False,NaN,/r/streetart/comments/b7e92j/brexcellent/,False,image,{'images': [{'source': {'url': 'https://previe...,NaN,False,NaN,NaN,False,498,NaN,{},NaN,NaN,True,False,False,streetart,t5_2qvd5,r/streetart,59230,public,NaN,https://b.thumbs.redditmedia.com/l5iQD_sD5Busm...,140.0,140.0,Brexcellent!,498,https://i.redd.it/7fi75cn5tap21.jpg,[],NaN,False,NaN,NaN,NaN
1,1,NaN,NaN,False,LIDER26,True,NaN,NaN,[],NaN,NaN,NaN,text,t2_xp37cb3,False,NaN,NaN,False,False,NaN,False,NaN,False,1.554034e+09,1.554034e+09,NaN,NaN,NaN,youtu.be,0,False,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",False,False,b7n12p,False,False,False,False,True,False,False,NaN,NaN,NaN,[],NaN,dark,text,False,{'oembed': {'provider_url': 'https://www.youtu...,"{'content': '&lt;iframe width=""600"" height=""33...",NaN,False,NaN,NaN,NaN,[],t3_b7n12p,False,0,0,NaN,False,NaN,/r/streetart/comments/b7n12p/graffiti_lide_yel...,False,rich:video,{'images': [{'source': {'url': 'https://extern...,NaN,False,NaN,NaN,False,7,{'oembed': {'provider_url': 'https://www.youtu...,"{'content': '&lt;iframe width=""600"" height=""33...",NaN,NaN,True,False,False,streetart,t5_2qvd5,r/streetart,59230,public,NaN,https://b.thumbs.redditmedia.com/ITNWfP8o6XmiG...,105.0,140.0,Graffiti - LIDE - Yellow Letters,7,https://youtu.be/2FCRgjrQ8rw,[],NaN,False,NaN,NaN,NaN


In [ ]:
  # dropping duplicte values, name had the highest value count so it seemed safest.
df.drop_duplicates(subset ="name", inplace = True) 

In [123]:
#oh no, I have hardly anything to work with.  I'll add the title and then have that text as well
df['selftext'].isnull().sum()

959

In [129]:
#make a smaller df with just the post text and the subreddit so I can appent both df together.
df_art =pd.DataFrame()
df_art['post'] =df['selftext']
df_art['title'] =df['title']
df_art['subreddit'] =df['subreddit']

In [ ]:
#export this so I can add it to the music dataframe.
df_art.to_csv('../data/dfs/art3cols_csv')

#### Now from these notebooks I have a dfs to begin modeling in notebook 3.